In [30]:
import joblib
import re
import string
import jieba
import pandas as pd
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm import tqdm
tqdm.pandas()
from transformers import PreTrainedTokenizerFast
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [31]:
# 请修改测试集文件名
test_dir = 'test.csv'

In [32]:
# 如测试集中不存在topic列，请使用 test = df[['content']]
df = pd.read_csv(test_dir)
test = df[['content', 'topic']]
#test = df[['content']]

In [33]:
def get_stopword():
    with open('baidu_stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f)
    stopwords.update(["年","月","日"])
    return stopwords

def wordopt_cn(text):
    # replace punctuation with space
    no_punct = re.sub(pattern, ' ', text)
    return str(no_punct)

def remove_stopwords(words):
    return (word for word in words if word not in stopwords)

def process_content(content):
    content = wordopt_cn(content)
    words = jieba.cut(content)
    words = remove_stopwords(words)
    content = ' '.join(words)
    content = re.sub(spaces_pattern, ' ', content)
    return content

# Precompile the regular expressions
punctuation = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏." + string.punctuation
pattern = re.compile('[%s]' % re.escape(punctuation))
spaces_pattern = re.compile(r'\s+')
stopwords = get_stopword()

In [34]:
test['original_content'] = test['content']
test['content'] = test['content'].progress_apply(process_content)

C:\Users\shens\AppData\Local\Temp\ipykernel_10144\2199731547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['original_content'] = test['content']
100%|██████████| 3405/3405 [00:11<00:00, 303.01it/s]
C:\Users\shens\AppData\Local\Temp\ipykernel_10144\2199731547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['content'] = test['content'].progress_apply(process_content)


In [35]:
def dummy(text):
    return text
#test=pd.read_csv('test_jieba.csv')
vectorizer = joblib.load('vectorizer.pkl')
model = joblib.load('ensemble.pkl')

In [36]:
LOWERCASE = False
VOCAB_SIZE = 100_000
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

dataset = Dataset.from_pandas(test[['content']])


def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i: i + 1000]["content"]


raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['content'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))
    
tf_test = vectorizer.transform(tokenized_texts_test)

100%|██████████| 3405/3405 [00:01<00:00, 1886.64it/s]


In [37]:
pred = model.predict(tf_test)
prob = model.predict_proba(tf_test)
test["pred"] = pred
test["prob"] = prob.max(axis=1)
test

C:\Users\shens\AppData\Local\Temp\ipykernel_10144\1750799718.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pred"] = pred


,content,topic,original_content,pred,prob
0,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,22,0.698144
1,当地 时间 11 16 习近平 主席 亚太经合组织 APEC 工商 领导人 峰会 发表 书面...,1,当地时间11月16日，习近平主席向亚太经合组织（APEC）工商领导人峰会发表书面演讲，强调我...,1,0.992717
2,亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科 尼 中心 国家 主席 习...,1,亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。国家主席习近平出席会议并发...,1,0.993817
3,习近平 出席 亚太经合组织 领导人 东道主 嘉宾 非正式 对话会 暨 工作 午宴 央视网 2...,1,习近平出席亚太经合组织领导人同东道主嘉宾非正式对话会暨工作午宴央视网2023年11月17日 ...,1,0.978806
4,中国 国家 主席 习近平 11 14 日应 美国 总统 拜登 邀请 赴美 中 美 领导人 会...,1,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1,0.668087
...,...,...,...,...,...
3400,从头到尾 来讲 一遍 惯例 先 谈谈 缅甸 北部 那场 风波 这场 风波 源头 藏匿在 缅甸...,23,从头到尾来讲一遍，咱们按照惯例，先谈谈缅甸北部的那场风波。这场风波的源头是藏匿在缅甸北部的一...,23,0.904904
3401,缅北 全线 激战 敏昂 莱 誓言 反击 冲突 扩大化 中 缅 边境 乱 几天 缅甸 内战 爆...,23,缅北全线激战，敏昂莱誓言反击，冲突可能扩大化，中缅边境乱不得这几天，缅甸内战的爆发，引起了外...,23,0.992973
3402,近日 缅甸 内战 再次 卷土重来 国际 社会 关注 这次 冲突 异常 纷乱 消息 传来 人 ...,23,近日，缅甸内战再次卷土重来，引起了国际社会广泛的关注。然而，这次的冲突异常纷乱，各种消息传来...,23,0.993701
3403,缅北 战火 越烧 越大 中方 做 两手 高层 去 缅甸 首都 清酒 半栩换种 角度 分析 故...,23,缅北战火越烧越大，中方做两手准备，高层直接去了缅甸首都清酒半栩换种角度分析故事，带你了解更有...,23,0.989968


In [38]:
diff = test[test['topic'] != test['pred']]
diff

,content,topic,original_content,pred,prob
0,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,22,0.698144
7,APEC 峰会 前 公使 提出 四点 核心 利益 中 美 会晤 看 美 表现 一带 一路 国...,1,APEC峰会前，我公使提出四点核心利益，中美是否会晤，得看美表现“一带一路”国际高峰论坛取得...,4,0.511553
40,2023 亚太经合 会 APEC 领袖 峰会 今 15 日于 美国 旧金山 登场 泰国 总理...,1,2023年亚太经合会APEC领袖峰会今(15)日于美国旧金山登场，泰国总理赛塔(Settha...,0,0.498137
43,焦点访谈 把舵 中美关系 领航 亚太 合作 央视网 2023 11 19 22 01 加载 ...,1,焦点访谈：把舵中美关系 领航亚太合作央视网2023年11月19日 22:01正在加载央视网消...,4,0.792356
60,亚太 国家 地区 领导人 13 抵达 美国 亚太经合组织 APEC 第三十次 领导人 非正式...,1,随着亚太一些国家和地区领导人从13日开始抵达美国，亚太经合组织（APEC）第三十次领导人非正...,4,0.731006
...,...,...,...,...,...
3096,2022 11 28 上午 9 时 神舟 十五号 载人 飞行 新闻 发布会 酒泉卫星发射中心...,21,2022年11月28日上午9时，神舟十五号载人飞行任务新闻发布会在酒泉卫星发射中心召开，中国...,19,0.925044
3099,2023 12 31 日晚 国家 主席 习近平 发表 二 〇 二 四年 新年贺词 2013 ...,21,2023年12月31日晚，国家主席习近平发表二〇二四年新年贺词。自2013年岁末起，习近平主...,19,0.490948
3127,神舟 十七号 发射成功 续写 飞天 梦想 建设 航天 强国 北京 时间 10 26 11 时...,21,神舟十七号发射成功：续写飞天梦想，建设航天强国。北京时间10月26日11时13分59秒，搭载...,20,0.946573
3132,神舟 十六号 出舱 多个 首次 神 十七 10 发射 航天员 杨利伟 7 国外 航天 执行 ...,21,神舟十六号出舱，有多个首次！神十七10月发射，有航天员杨利伟？7月，国外的航天任务执行可能最...,20,0.714251


In [39]:
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9750367107195301
Precision: 0.9752436834777642
Recall: 0.9750367107195301
F1: 0.9750324678322444


In [41]:
low_prob = test[test['prob'] < 0.5]
low_prob

,content,topic,original_content,pred,prob
40,2023 亚太经合 会 APEC 领袖 峰会 今 15 日于 美国 旧金山 登场 泰国 总理...,1,2023年亚太经合会APEC领袖峰会今(15)日于美国旧金山登场，泰国总理赛塔(Settha...,0,0.498137
82,美国 已 中 俄 发出 邀请 普京 本人 去不去 俄罗斯 会 悬念 留 亚太经合组织 APE...,1,美国已向中俄发出邀请，普京本人去不去，俄罗斯会把悬念留到最后今年的亚太经合组织（APEC）峰...,3,0.366565
148,微软 采购 核聚变 初创 公司 Helion 电力 Helion 预计 2028 年前 上线...,2,微软将采购由核聚变初创公司Helion产生的电力，Helion预计2028年前上线50兆瓦核...,0,0.380416
252,2023 新德里 G20 领导人 峰会 成员国 名单 特邀 成员国 名单 印度 拟定 20 ...,3,2023年在新德里举行的G20领导人峰会成员国名单以及特邀成员国名单，印度已经拟定出来，除了...,3,0.460188
261,印度 新德里 G20 峰会 落幕 国际 政坛 落幕 各国 领导人 展现 风采 机会 英国 头...,3,印度新德里G20峰会的落幕，不仅是国际政坛的落幕，也是各国领导人展现风采的机会。英国头号夫人...,3,0.444316
...,...,...,...,...,...
3061,2023 神舟 16 号 发射成功 观后感 三 二 点火 火团 喷涌 而出 小小的 屏幕 画...,21,2023年神舟16号发射成功观后感“三、二、一，点火”。火团喷涌而出。望着小小的屏幕，我看到...,19,0.304691
3075,神舟 十六号 发射 全过程 47002023 05 30 11 07 43 未经 作者 授权...,21,神舟十六号_发射全过程47002023-05-30 11:07:43未经作者授权，禁止转载9...,21,0.274267
3099,2023 12 31 日晚 国家 主席 习近平 发表 二 〇 二 四年 新年贺词 2013 ...,21,2023年12月31日晚，国家主席习近平发表二〇二四年新年贺词。自2013年岁末起，习近平主...,19,0.490948
3118,点击 右上角 微信 好友 朋友圈 请 浏览器 分享 功能 分享 导航 时政 国际 时评 理论...,21,点击右上角微信好友朋友圈请使用浏览器分享功能进行分享全部导航时政国际时评理论文化科技教育经济...,21,0.443351
